In [1]:
import sys
import json
from src.data import generate_dataset

In [2]:
with open('./config/data_params.json') as f:
    data_params = json.load(f)
with open('./config/sample_params.json') as f:
    sample_params = json.load(f)

# Cfg variables
raw_data_path = data_params['raw_data_path']
processed_data_path = data_params['dehydrated_data_path']
twarc_path = data_params['twarc_path']
rehydrated_df_path = data_params['rehydrated_df_path']
id_column = data_params['id_column']
day = data_params['day']

seed = sample_params['seed']
sample_size = sample_params['sample_size']

seed = sample_params['seed']
sample_size = sample_params['sample_size']



# # Update dataset from some date (set in data_params) to today
# generate_dataset.download_latest_datasets(raw_data_path, day)
# # sample raw data of tweet ids, rehydrate them (enrich them with tweet contents)
# print(generate_dataset.rehydrate_tweets(raw_data_path, processed_data_path, seed, sample_size, id_column, twarc_path, day))

In [4]:
import pandas as pd

In [7]:
last_day = '2020-03-22'
to_day = '2020-07-31'

In [8]:
pd.date_range(last_day, to_day, freq='d')

DatetimeIndex(['2020-03-22', '2020-03-23', '2020-03-24', '2020-03-25',
               '2020-03-26', '2020-03-27', '2020-03-28', '2020-03-29',
               '2020-03-30', '2020-03-31',
               ...
               '2020-07-22', '2020-07-23', '2020-07-24', '2020-07-25',
               '2020-07-26', '2020-07-27', '2020-07-28', '2020-07-29',
               '2020-07-30', '2020-07-31'],
              dtype='datetime64[ns]', length=132, freq='D')